# Swin Transfomer
We learned a few lessons from the previous experiment. First, and primarily, a larger model doesn't always mean better. We used the large imageNet transformer model, but that didn't actually lead to good predictions. Also, I think I screwed up a bunch of stuff, and the large model is taking way too long to train for me to iterate at all. We are going to switch to a smaller model. We are going to follow the ViTMAE method, use masked auto encodings for pretraining in a self-supervised manner on the entire training set, then fine tune on the prediction task. Guide found here: https://github.com/huggingface/transformers/blob/main/examples/pytorch/image-pretraining/run_mae.py 

This notebook has the fine-tuning process, fine tuning on Binary CE Loss.

In [1]:
# Mount into drive
from google.colab import drive
drive.mount("/content/drive")
!cp '/content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/CheXpert-v1.0-small.zip' /content/
!mkdir CheXpert
!unzip '/content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/CheXpert-v1.0-small.zip' -d /content/CheXpert
import sys
sys.path.append('/content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/tyler')
sys.path.append('/content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation')
sys.path.append('/content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/models')

Streaming output truncated to the last 5000 lines.
  inflating: /content/CheXpert/CheXpert-v1.0-small/train/patient21643/study3/view2_lateral.jpg  
  inflating: /content/CheXpert/CheXpert-v1.0-small/train/patient21643/study3/view1_frontal.jpg  
   creating: /content/CheXpert/CheXpert-v1.0-small/train/patient21643/study1/
  inflating: /content/CheXpert/CheXpert-v1.0-small/train/patient21643/study1/view1_frontal.jpg  
   creating: /content/CheXpert/CheXpert-v1.0-small/train/patient21643/study2/
  inflating: /content/CheXpert/CheXpert-v1.0-small/train/patient21643/study2/view1_frontal.jpg  
   creating: /content/CheXpert/CheXpert-v1.0-small/train/patient63218/
   creating: /content/CheXpert/CheXpert-v1.0-small/train/patient63218/study1/
  inflating: /content/CheXpert/CheXpert-v1.0-small/train/patient63218/study1/view1_frontal.jpg  
   creating: /content/CheXpert/CheXpert-v1.0-small/train/patient08046/
   creating: /content/CheXpert/CheXpert-v1.0-small/train/patient08046/study1/
  inflatin

In [2]:
# imports
!pip install transformers
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

import torch
print(torch.cuda.is_available())

from transformers import ConvNextFeatureExtractor, ConvNextForImageClassification
from transformers import TrainingArguments, Trainer
from torchvision.transforms import RandomHorizontalFlip, RandomResizedCrop
from torchvision.transforms.functional import InterpolationMode

from sklearn.model_selection import train_test_split

     |████████████████████████████████| 4.0 MB 30.3 MB/s 
     |████████████████████████████████| 77 kB 8.5 MB/s 
     |████████████████████████████████| 596 kB 87.1 MB/s 
     |████████████████████████████████| 6.6 MB 83.1 MB/s 
     |████████████████████████████████| 880 kB 88.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=74138d27313603833032f7566bac9dbce258315f130646d4f39b16884411943a
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
True


In [3]:
!pip install libauc
from dataloader import *
from utils import *
from trainer import *

     |████████████████████████████████| 462 kB 59.4 MB/s 


In [4]:
# CONSTANTS
FEATURE_EXTRACTOR_NAME = 'facebook/convnext-tiny-224'
VIT_MODEL_NAME = 'facebook/convnext-tiny-224'
TRAIN_SPLIT = 0.8
BATCH_SIZE = 48
LEARNING_RATE = 1.5e-3
LR_SCHEDULER_TYPE = "cosine"
WEIGHT_DECAY = 0.05
WARMUP_RATIO = 0.05
LOGGING_STRATEGY = "steps"
LOGGING_STEPS = 10
FP16 = True
EPOCHS = 10
EVALUATION_STRATEGY = "steps"
EVAL_STEPS = 250
OUTPUT_DIR = '/content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/convnext-chexpert-fine-tuned-output'
REMOVE_UNUSED_COLUMNS = False
GRAD_ACCUM_STEPS = 5
DATALOADER_NUM_WORKERS = 4

In [5]:
feature_extractor = ConvNextFeatureExtractor.from_pretrained(FEATURE_EXTRACTOR_NAME, image_mean=[0.485, 0.456, 0.406], image_std=[0.229, 0.224, 0.225])

Downloading:   0%|          | 0.00/266 [00:00<?, ?B/s]

In [6]:
# set up our transforms
transforms = [
    RandomResizedCrop(feature_extractor.size, scale=(0.2, 1.0), interpolation=InterpolationMode.BICUBIC),
    RandomHorizontalFlip(),
]

In [7]:
np.random.seed(42)
train_df = pd.read_csv("/content/CheXpert/CheXpert-v1.0-small/train.csv")
train_df, eval_df = train_test_split(train_df, train_size=TRAIN_SPLIT)

train_dataset = ChexpertViTDataset("/content/CheXpert", train_df, feature_extractor, transforms=transforms, classes=COMPETITION_TASKS,
    uncertainty_method="smooth", smoothing_lower_bound=0.55, smoothing_upper_bound=0.85, use_frontal = False)
eval_dataset = ChexpertViTDataset("/content/CheXpert", eval_df, feature_extractor, classes=COMPETITION_TASKS,
    uncertainty_method="smooth", smoothing_lower_bound=0.55, smoothing_upper_bound=0.85, use_frontal = False)

In [8]:
model = ConvNextForImageClassification.from_pretrained(
    VIT_MODEL_NAME,
    num_labels=len(train_dataset.labels),
    ignore_mismatched_sizes = True
).to("cuda")

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/109M [00:00<?, ?B/s]

Some weights of ConvNextForImageClassification were not initialized from the model checkpoint at facebook/convnext-tiny-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# set up training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    evaluation_strategy=EVALUATION_STRATEGY,
    num_train_epochs=EPOCHS,
    fp16=FP16,
    eval_steps = EVAL_STEPS,
    learning_rate=LEARNING_RATE,
    remove_unused_columns=REMOVE_UNUSED_COLUMNS,
    report_to="tensorboard",
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    logging_strategy=LOGGING_STRATEGY,
    logging_steps=LOGGING_STEPS,
    dataloader_num_workers=DATALOADER_NUM_WORKERS
)

In [10]:
trainer = MultiLabelTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

Using amp half precision backend


In [11]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 178731
  Num Epochs = 10
  Instantaneous batch size per device = 48
  Total train batch size (w. parallel, distributed & accumulation) = 240
  Gradient Accumulation steps = 5
  Total optimization steps = 7440


Step,Training Loss,Validation Loss,Accuracy,F1,Auc Atelectasis,Auc Cardiomegaly,Auc Consolidation,Auc Edema,Auc Pleural effusion,Average Auc
250,0.461700,0.444984,0.344314,0.493059,0.687510,0.786927,0.662453,0.824673,0.845588,0.761430
500,0.456400,0.439937,0.342166,0.490841,0.687185,0.800174,0.670330,0.828237,0.853468,0.767879
750,0.487900,0.427786,0.360271,0.492564,0.702429,0.818333,0.675088,0.836506,0.859125,0.778296
1000,0.445100,0.430045,0.355571,0.456861,0.697730,0.827399,0.674435,0.839304,0.861512,0.780076
1250,0.444500,0.435773,0.356444,0.502772,0.697698,0.823378,0.671460,0.838332,0.861553,0.778484
1500,0.449400,0.425588,0.361726,0.494903,0.704744,0.822796,0.679824,0.837012,0.861821,0.781239
1750,0.442200,0.426365,0.357384,0.475104,0.705517,0.831121,0.684110,0.838650,0.862455,0.784371
2000,0.429500,0.422861,0.363942,0.497994,0.702572,0.828736,0.681171,0.844284,0.862755,0.783903
2250,0.425900,0.431133,0.357093,0.462236,0.705497,0.815141,0.681507,0.842729,0.863124,0.781600


***** Running Evaluation *****
  Num examples = 44683
  Batch size = 48
***** Running Evaluation *****
  Num examples = 44683
  Batch size = 48
Saving model checkpoint to /content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/convnext-chexpert-fine-tuned-output/checkpoint-500
Configuration saved in /content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/convnext-chexpert-fine-tuned-output/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/convnext-chexpert-fine-tuned-output/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 44683
  Batch size = 48
***** Running Evaluation *****
  Num examples = 44683
  Batch size = 48
Saving model checkpoint to /content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/convnext-chexpert-fine-tuned-output/checkpoint-1000
Configuration saved in /content/drive/MyDrive/Georgia Tec

KeyboardInterrupt: ignored

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 44683
  Batch size = 48


{'epoch': 10.0,
 'eval_AUC_Atelectasis': 0.7171348778722131,
 'eval_AUC_Cardiomegaly': 0.8217412982534597,
 'eval_AUC_Consolidation': 0.6951434345294569,
 'eval_AUC_Edema': 0.8514513065825018,
 'eval_AUC_Pleural Effusion': 0.8712694779990864,
 'eval_accuracy': 0.36976031152787414,
 'eval_average_auc': 0.7913480790473436,
 'eval_f1': 0.5488138686131386,
 'eval_loss': 0.43087852001190186,
 'eval_runtime': 323.5241,
 'eval_samples_per_second': 138.113,
 'eval_steps_per_second': 2.878}

In [ ]:
trainer.save_state()

In [ ]:
trainer.save_model()

Saving model checkpoint to /content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/convnext-chexpert-fine-tuned-output
Configuration saved in /content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/convnext-chexpert-fine-tuned-output/config.json
Model weights saved in /content/drive/MyDrive/Georgia Tech/CS 7643 Deep Learning/chestx_interpretation/convnext-chexpert-fine-tuned-output/pytorch_model.bin


In [12]:
validation_df = pd.read_csv("/content/CheXpert/CheXpert-v1.0-small/valid.csv")
valid_dataset = ChexpertViTDataset("/content/CheXpert", validation_df, feature_extractor, classes=COMPETITION_TASKS,
    uncertainty_method="smooth", smoothing_lower_bound=0.55, smoothing_upper_bound=0.85, use_frontal = False)

In [14]:
trainer.evaluate(eval_dataset=valid_dataset)

***** Running Evaluation *****
  Num examples = 234
  Batch size = 48


Step,Training Loss,Validation Loss,Accuracy,F1,Auc Atelectasis,Auc Cardiomegaly,Auc Consolidation,Auc Edema,Auc Pleural effusion,Average Auc
250,0.461700,0.444984,0.344314,0.493059,0.687510,0.786927,0.662453,0.824673,0.845588,0.761430
500,0.456400,0.439937,0.342166,0.490841,0.687185,0.800174,0.670330,0.828237,0.853468,0.767879
750,0.487900,0.427786,0.360271,0.492564,0.702429,0.818333,0.675088,0.836506,0.859125,0.778296
1000,0.445100,0.430045,0.355571,0.456861,0.697730,0.827399,0.674435,0.839304,0.861512,0.780076
1250,0.444500,0.435773,0.356444,0.502772,0.697698,0.823378,0.671460,0.838332,0.861553,0.778484
1500,0.449400,0.425588,0.361726,0.494903,0.704744,0.822796,0.679824,0.837012,0.861821,0.781239
1750,0.442200,0.426365,0.357384,0.475104,0.705517,0.831121,0.684110,0.838650,0.862455,0.784371
2000,0.429500,0.422861,0.363942,0.497994,0.702572,0.828736,0.681171,0.844284,0.862755,0.783903
2250,0.425900,0.431133,0.357093,0.462236,0.705497,0.815141,0.681507,0.842729,0.863124,0.781600
2276,0.443200,0.422080,0.431624,0.434783,0.820455,0.831724,0.873737,0.919753,0.935696,0.876273


{'eval_AUC_Atelectasis': 0.8204545454545454,
 'eval_AUC_Cardiomegaly': 0.8317239546420979,
 'eval_AUC_Consolidation': 0.8737373737373737,
 'eval_AUC_Edema': 0.9197530864197531,
 'eval_AUC_Pleural Effusion': 0.9356957726338367,
 'eval_accuracy': 0.43162393162393164,
 'eval_average_auc': 0.8762729465775214,
 'eval_f1': 0.4347826086956523,
 'eval_loss': 0.42207974195480347}